In [2]:
import pandas as pd

In [3]:
df = pd.read_csv(r"DS_CentraleSupelec_ST42021/DS_CentraleSupelec_train.csv")

In [11]:
df

,userId,age,gender,registrationDate,languageCode,countryCode,emailContactable,postalContactable,postalCode,numberChildren,...,detergentType,MrPropreTrier,AntikalTrier,ArielTrier,DashTrier,scentLover,petOwner,ecoInterest,closestShop,washDishes
0,use_XbA1FTDcCrTMNTHK1851TzjyPMP,24,F,2019-10-14T17:33:37Z,fr,FRA,True,True,68540,1,...,Liquid,NaN,NaN,NaN,NaN,NonUser,Yes,NaN,NaN,Auto
1,use_3WHgsMVGSg5MHG2zja91TzdfmY2,67,F,2017-07-25T07:00:31Z,fr,FRA,True,True,17390,3,...,Pods,Known Trier,Known Trier,Known Trier,NaN,NaN,No,High,NaN,Auto
2,use_2SisOiR8QwDaHI4svm11TzcvK7V,45,F,2015-06-25T00:00:00Z,fr,FRA,True,True,30200,1,...,Liquid,NaN,NaN,Known Trier,Known Trier,NonUser,Yes,High,NaN,Auto
3,use_W0T1LmfKaQPJYD1RTWh1Tzdc69H,35,F,2018-01-31T07:24:39Z,fr,FRA,True,True,61400,2,...,Pods,NaN,NaN,Known Trier,Known Trier,Low,Yes,Medium,NaN,Hand
4,use_KwYWZ7UMl4veveOaPGK1TzddtmI,38,F,2016-01-26T11:42:17Z,fr,FRA,True,True,30620,0,...,Liquid,Known Trier,Known Trier,Known Trier,Known Trier,NaN,Yes,Low,NaN,Hand
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,use_ICSyEvCcRNPpQeBdWcM1TzdczV8,53,F,2015-03-11T20:03:16Z,fr,FRA,True,True,77240,4+,...,Pods,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,Auto
11996,use_Q5DNYeSmG7g6VeMV4n71TzeTRRz,43,F,2015-10-10T19:16:19Z,fr,FRA,True,True,45120,0,...,Liquid,NaN,NaN,NaN,NaN,Low,Yes,Medium,NaN,Hand
11997,use_rEoce5pc0MlKpndfzNj1Tze3J1L,42,F,2015-05-08T02:59:34Z,fr,FRA,True,True,35170,3,...,Liquid,NaN,NaN,NaN,NaN,NaN,Yes,High,NaN,Hand
11998,use_0AuDL2wOJHy9I16zSC21TzeHRMo,39,M,2019-05-20T11:59:02Z,fr,FRA,True,True,45160,1,...,Liquid,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,Auto


We can remove the `languageCode` and `countryCode` column as they are constant

In [21]:
df[~df.languageCode.isin(['fr', 'FR'])].empty and df[~df.countryCode.eq('FRA')].empty

True

We will also remove the `userId`, `registrationDate`, `emailContactable` and `postalContactable` as they are irrelevant